Copyright **`(c)`** 2023 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

In [1]:
from itertools import product
from random import random, randint, shuffle, seed, choice, randint
import numpy as np
from scipy import sparse
from functools import reduce
from collections import namedtuple
from queue import PriorityQueue, SimpleQueue, LifoQueue
from copy import copy

In [2]:
def make_set_covering_problem(num_points, num_sets, density):
    """Returns a sparse array where rows are sets and columns are the covered items"""
    seed(num_points*2654435761+num_sets+density)
    sets = sparse.lil_array((num_sets, num_points), dtype=bool)
    for s, p in product(range(num_sets), range(num_points)):
        if random() < density:
            sets[s, p] = True
    for p in range(num_points):
        sets[randint(0, num_sets-1), p] = True
    return sets

# Halloween Challenge

Find the best solution with the fewest calls to the fitness functions for:

* `num_points = [100, 1_000, 5_000]`
* `num_sets = num_points`
* `density = [.3, .7]` 

In [3]:
NUM_SETS = 5_000
DENSITY = .3
PROBLEM_SIZE = NUM_POINTS = NUM_SETS
x = make_set_covering_problem(NUM_POINTS, NUM_SETS, DENSITY)
print("Element at row=42 and column=42:", x[42, 42])

Element at row=42 and column=42: True


In [4]:
def fitness2(state):
    cost = sum(state)
    valid = np.sum(
        reduce(
            np.logical_or,
            [np.array(x.getrow(i).toarray()[0]) for i, t in enumerate(state) if t],
            np.array([False for _ in range(PROBLEM_SIZE)]),
        )
    )
    return valid, -cost

fitness = fitness2

La funzione tweak fornita dal professore prende e toglie randomicamente i set dalla soluzione.

Un modo intelligente per scalare la collina sarebbe:
 * aggiungere un set che copre nuovi punti (perche mi avvicina a goal)
 * scambiare un set che copre dei punti gia coperti con un altro set che copre nuovi punti
 * rimuovere un set che copre dei punti gia coperti (ovvero che non mi avvicina a goal)

In [5]:
def tweak(state, ffid):
    new_state = copy(state)
    # invece che prendere uno a caso (weak causality), prendo uno che potenzialmente potrebbe avvicinarmi alla soluzione
    # filtro prendo per primi i set che all'indice in cui é vuoto hanno true e hanno più true

    #index = randint(0, PROBLEM_SIZE - 1)
    #new_state[index] = not new_state[index]
    rows_with_true_at_first_false_index = np.where(np.array(x[:, [ffid]].toarray()))[0]

    # Trova le righe che non sono presenti in new_state
    # in teoria posso toglierlo perche tanto non ci sono righe che hanno true in ffid in new_state
    rows_not_in_new_state = np.setdiff1d(rows_with_true_at_first_false_index, np.where(new_state))

    # Seleziona casualmente una riga tra quelle rimanenti
    if len(rows_not_in_new_state) > 0:
        random_row_index = np.random.choice(rows_not_in_new_state)
        #print("Riga casuale con il primo indice di colonna True non presente in new_state:")
        new_state[random_row_index] = not new_state[random_row_index]
    else:
        print("Nessuna riga soddisfa i criteri specificati.")
        random_row_index = np.random.choice(np.where(new_state))
        new_state[random_row_index] = not new_state[random_row_index]
        #in questo caso dovrò togliere un elemento da new_state, per ora uno a caso 
        # ma potrei vedere per uno che non mi toglie troppi punti coperti
    return new_state

In [6]:
current_state = [choice([False, False, False, False, False, False]) for _ in range(NUM_SETS)]
print(fitness(current_state))

cnt = 0
diff = 0
first_false_index = 0
fit_cnt = 0

for step in range(10_000):
    new_state = tweak(current_state, first_false_index)
    fn = fitness(new_state)
    fc = fitness(current_state)
    fit_cnt += 1
    if fn >= fc:
        #diff potrebbe servirmi come criterio per la tweak
        #diff = (fn[0]-fc[0],  fn[1] - fc[1])
        #print(diff)
        # per ogni volta che mi avvicino (e quindi non in tutte le iterazioni in tweak)
        # calcolo il 1° indice false da riempire
        #sto calcolo potrei riciclarlo da fitness ritornandolo facendolo insieme alla sum per accorciare i tempi?
        result_of_reduce = reduce(
            np.logical_or,
            [np.array(x.getrow(i).toarray()[0]) for i, t in enumerate(new_state) if t],
            np.array([False for _ in range(PROBLEM_SIZE)]),
        )

        first_false_index = None
        for i, value in enumerate(result_of_reduce):
            #print(i, value)
            if not value:
                first_false_index = i
                print(first_false_index)
                break

        # Trova l'indice del primo valore falso (False)
        if first_false_index is None:
            print("Tutti i valori sono veri.")
            current_state = new_state
            break

        cnt += 1
        current_state = new_state
        #print(fitness(current_state))

print(f"Steps: {cnt}")
print(f"Fitness calls: {fit_cnt}")

(0, 0)
1
2
7
8
16
22
66
135
158
195
215
422
452
800
828
919
1499
3019
3715
3809
Tutti i valori sono veri.
20
21


In [7]:
#current_state
red = reduce(
            np.logical_or,
            [np.array(x.getrow(i).toarray()[0]) for i, t in enumerate(current_state) if t],
            np.array([False for _ in range(PROBLEM_SIZE)]),
        )
print(f"solution lenght: {np.sum(current_state)}")
print(f"solution: {red}")
if np.all(red): 
    print("Tutti i valori sono veri.")
#for i in range(len(current_state)):
#    if current_state[i]:
#        print(i)
        #print(np.array(x.getrow(i).toarray()[0]))

solution lenght: 21
solution: [ True  True  True ...  True  True  True]
Tutti i valori sono veri.
317
634
656
674
853
1063
1388
1454
1607
1783
1865
1990
1991
3763
3792
3818
4127
4473
4541
4747
4983
